In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
# from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import metrics
import scattertext as st
from pprint import pprint

In [2]:
## Initial Spam Classification with NB

In [3]:
# filename = 'sms_data'
# sms_df = pd.read_csv(filename, header=None, sep='	', names=['class_sms', 'sms'])
# sms_df['sms'] = sms_df['sms'].str.replace(r'\d+', '')

# sms_df['sms'] = sms_df['sms'].apply(lambda x: x.split())
# sms_df['sms'] = sms_df['sms'].apply(lambda x: [word for word in x if len(word) > 2])
# sms_df['sms'] = sms_df['sms'].apply(lambda x: ', '.join(x))
# # print(sms_df['sms'])

# X = sms_df.sms
# y = sms_df.class_sms

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# vect = CountVectorizer()
# X_train_dtm = vect.fit_transform(X_train) 
# # print(vect.get_feature_names())
# X_test_dtm = vect.transform(X_test)

# nb = MultinomialNB()
# nb.fit(X_train_dtm, y_train)
# y_pred_class = nb.predict(X_test_dtm)  
# false_positive = X_test[(y_test == 'spam') & (y_pred_class == 'ham')]
# false_negative = X_test[(y_test == 'ham') & (y_pred_class == 'spam')]
# # print(false_positive)
# # print(false_negative)

# # print(y_test.value_counts())  # examine the class distribution of the testing set (using a Pandas Series method)
# # print(y_test.value_counts().head(1) / len(y_test)) 
# print('General Accuracy:', metrics.accuracy_score(y_test, y_pred_class)) # 0.9885
# confusion_matrix = metrics.confusion_matrix(y_test, y_pred_class)
# false_positive_rate = confusion_matrix[1][0] / (confusion_matrix[1][0] + confusion_matrix[1][1])
# print('False Positive Rate:', false_positive_rate*100) # 5.405% <-- terrible!


In [4]:
# X_train_tokens = vect.get_feature_names()
# ham_token_count = nb.feature_count_[0, :]
# spam_token_count = nb.feature_count_[1, :]
# tokens = pd.DataFrame({'token': X_train_tokens, 'ham': ham_token_count, 'spam': spam_token_count})
# tokens['ham'] = tokens.ham + 1
# tokens['spam'] = tokens.spam + 1
# tokens['ham'] = tokens.ham / nb.class_count_[0]
# tokens['spam'] = tokens.spam / nb.class_count_[1]
# tokens['ham_ratio'] = tokens.ham / tokens.spam
# tokens['spam_ratio'] = tokens.spam / tokens.ham
# print(tokens.sort_values('ham_ratio', ascending=False).head(10))  # top 10 tokens predictive for 5-star
# print(tokens.sort_values('spam_ratio', ascending=False).head(10))  # top 10 tokens predictive for 5-star
# print(tokens.sort_values('one_star_ratio', ascending=False).head(10))  # top 10 tokens predictive for 1-star

In [5]:
## The 2 blocks of code below produce the HTML text visualization

In [6]:
# nlp = st.WhitespaceNLP.whitespace_nlp
# corpus = st.CorpusFromPandas(sms_df, 
#                               category_col='class_sms', 
#                               text_col='sms',
#                               nlp=nlp).build()
# term_freq_df = corpus.get_term_freq_df()
# term_freq_df['spam'] = corpus.get_scaled_f_scores('spam')
# pprint(list(term_freq_df.sort_values(by='spam', ascending=False).index[:10])) # words most associated with spam
# term_freq_df['ham'] = corpus.get_scaled_f_scores('ham')
# pprint(list(term_freq_df.sort_values(by='ham', ascending=False).index[:10]))  # words most associated with ham

In [7]:
# html = st.produce_scattertext_explorer(corpus,
#           category='spam',
#           category_name='spam',
#           not_category_name='ham',
#           width_in_pixels=1000,
#           )
# open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))

In [8]:
## Post-Naive Bayes:
# Tokenization (playing around with data, reading RP/posts to get ideas )
# Feature Selection (information gain)
# Evaluation

In [14]:
filename = 'sms_data'
sms_df = pd.read_csv(filename, header=None, sep='	', names=['class_sms', 'sms'])
X = sms_df.sms
y = sms_df.class_sms
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
stopword = stopwords.words('english')[0:80]
print(stopword)
vect = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b', lowercase=False, ngram_range=(1,2), stop_words=stopword)
X_train_dtm = vect.fit_transform(X_train) 
X_test_dtm = vect.transform(X_test)

nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)  
print('General Accuracy:', metrics.accuracy_score(y_test, y_pred_class)) # 0.9885
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_class)
false_positive_rate = confusion_matrix[1][0] / (confusion_matrix[1][0] + confusion_matrix[1][1])
print('False Positive Rate:', false_positive_rate*100) # 5.405% <-- terrible!
# print (classification_report(y_test, y_pred_class))
spam_token_count = nb.feature_count_[1, :] 
X_train_tokens = vect.get_feature_names()
tokens = pd.DataFrame({'token':X_train_tokens, 'spam':spam_token_count})
tokens['spam'] = tokens.spam + 1
tokens['spam'] = tokens.spam / nb.class_count_[1]
print(tokens.sort_values('spam', ascending=False))
        


# vect = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b', lowercase=False, ngram_range=(1,2), tokenizer=TreebankWordTokenizer().tokenize)
# print(vect.get_feature_names())
# print(len(vect.get_feature_names()))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to']
General Accuracy: 0.990667623833
False Positive Rate: 6.48648648649
           spam          token
871    0.279359              2
16796  0.270463           call
1511   0.192171              4
3918   0.186833           Call
30503  0.177936             on
30040  0.172598            now
21858  0.163701           from
5232   0.153025           FREE
12315

In [10]:
# nlp = st.WhitespaceNLP.whitespace_nlp
# corpus = st.CorpusFromPandas(sms_df, 
#                               category_col='class_sms', 
#                               text_col='sms',
#                               nlp=nlp).build()
# term_freq_df = corpus.get_term_freq_df()
# term_freq_df['spam'] = corpus.get_scaled_f_scores('spam')
# pprint(list(term_freq_df.sort_values(by='spam', ascending=False).index[:10])) # words most associated with spam
# term_freq_df['ham'] = corpus.get_scaled_f_scores('ham')
# pprint(list(term_freq_df.sort_values(by='ham', ascending=False).index[:10]))  # words most associated with ham
# html = st.produce_scattertext_explorer(corpus,
#           category='spam',
#           category_name='spam',
#           not_category_name='ham',
#           width_in_pixels=1000,
#           )
# open("Convention-Visualization.html", 'wb').write(html.encode('utf-8'))